In [1]:
%autosave 60

Autosaving every 60 seconds


# The $3 \times 3$ Sliding Puzzle

<img src="8-puzzle.png">

The picture above shows an instance of the $3 \times 3$ 
<a href="https://en.wikipedia.org/wiki/Sliding_puzzle">sliding puzzle</a>:
There is a board of size $3 \times 3$ with 8 tiles on it. These tiles are numbered with digits from the set $\{1,\cdots, 8\}$.  As the the $3 \times 3$ board has an area of $9$ but there are only $8$ tiles, there is an empty square on the board.  Tiles adjacent to the empty square can be moved into the square, thereby emptying the space that was previously occupied by theses tiles.  The goal of the $3 \times 3$ puzzle is to transform the state shown on the left of the picture above into the state shown on the right.

In order to get an idea of the sliding puzzle, you can play it online at <a href="http://mypuzzle.org/sliding">http://mypuzzle.org/sliding</a>.

## Computing a Path 

The functions given below are identical to the functions that we have used to solve the shunting-yard problem from the previous exercise.

In [2]:
def pathProduct(P, R):
    "Attach all possible pairs from R at the end of the paths from P."
    return { L + (b,) for L in P for (a, b) in R 
                      if L[-1] == a and not b in L 
           }

In [3]:
def arb(S):
    "Return an arbitrary element from the set S but do not remove it."
    for x in S:
        return x

The sliding puzzle has $9! = 362\,880$ different states and the transition relation $R$ has a size of $846\,720$ pairs.  Keeping the relation $R$ in memory is going to be slow.  Instead, we will implement a function $\texttt{nextStates}(S)$ that takes a state $S$ as input and computes the set of all states that can be reached from $S$.  For any given state $S$ the set $\texttt{nextStates}(S)$ has at most four elements.  Accordingly I had to change the definition of the function <tt>findPath</tt>.  This function is now called as $\texttt{findpath}(\texttt{start},\texttt{goal}, \texttt{nextStates})$ where <tt>start</tt> and <tt>goal</tt> are states and $\texttt{nextStates}$ is a function.   It implements the *breadth first search* algorithm.

In [4]:
def findPath(start, goal, nextStates):
    count    = 0
    Paths    = { (start,) }
    States   = { start }   # states known to be reachable
    Explored = {}          # states that have already been explored
    while States != Explored:
        Explored = States.copy()     # avoid aliasing bug
        Paths    = { P + (s,) for P in Paths
                              for s in nextStates(P[-1])
                              if  s not in States
                   }
        States  |= { P[-1] for P in Paths }
        count   += 1
        print("%3s: Number of states = %6s" % (count, len(States)))
        if goal in States: 
            return arb({ P for P in Paths if P[-1] == goal })

## Problem Specific Code

We will represent states as tuples of tuples.  For example, the start state that is shown in the picture at the beginnning of this notebook is represented as follows:

In [5]:
start = ((8, 0, 6),
         (5, 4, 7),
         (2, 3, 1)
        )

Note that the empty tile is represented by the digit $0$.  

Similarly, the goal state is defined as:

In [6]:
goal = ((0, 1, 2),
        (3, 4, 5),
        (6, 7, 8)
       )

The function $\texttt{findZero}(S)$ takes a state $S$ and returns a pair $(r, c)$ that specifies the row and the column of the blank in the state $S$.

In [7]:
def findZero(S):
    M = {0, 1, 2}
    P = {(r, c) for r in M
                for c in M 
                if S[r][c] == 0
        }
    return arb(P)

In [8]:
findZero(start)

(0, 1)

In [9]:
findZero(goal)

(0, 0)

We represent states as tuples in order to be able to insert them into sets.  However, as ss tuples are immutable, we need to be able to convert them to lists in order to change them.
The function $\texttt{listOfLists}(S)$ takes a state $S$ and transforms it into a list of lists.

In [10]:
def listOfLists(S):
    "Transform a tuple of tuples into a list of lists."
    return [ [x for x in row] for row in S ]

In [11]:
listOfLists(start)

[[8, 0, 6], [5, 4, 7], [2, 3, 1]]

As lists can not be inserted into sets, we also need a function that takes a list of list and transforms it into a tuple of tuple.

In [12]:
def tupleOfTuples(S):
    "Transform a list of lists into a tuple of tuples."
    return tuple(tuple(x for x in row) for row in S)

In [13]:
tupleOfTuples([[8, 0, 6], [5, 4, 7], [2, 3, 1]])

((8, 0, 6), (5, 4, 7), (2, 3, 1))

**Exercise 1**: Implement a function $\texttt{moveUp}(S, r, c)$ that computes the state that results from moving the tile below the blank space **up** in state $S$.  The variables $r$ and $c$ specify the location of the *row* and *column* of the blank tile.  Therefore we have $S[r][c] = 0$.

In your implementation you may assume that there is indeed a tile below the blank space, i.e. we have $r < 2$.  

In [14]:
def moveUp(S, r, c):
    "Move the tile below the blank up."
    T = listOfLists(S)
    T[r][c], T[r+1][c] = T[r+1][c], 0
    return tupleOfTuples(T)

In [15]:
moveUp(start, 0, 0)

((5, 0, 6), (0, 4, 7), (2, 3, 1))

**Exercise 2**: Implement a function $\texttt{moveDown}(S, r, c)$ that computes the state that results from moving the tile below the blank space **down** in state $S$.  The variables $r$ and $c$ specify the location of the *row* and *column* of the blank tile.  Therefore we have $S[r][c] = 0$.

In your implementation you may assume that there is indeed a tile above the blank space, i.e. we have $r > 0$.  

In [16]:
def moveDown(S, r, c):
    "Move the tile above the blank down."
    T = listOfLists(S)
    T[r][c], T[r-1][c] = (T[r-1][c], 0)
    return tupleOfTuples(T)

**Exercise 3:**
Similarly to the previous exercise, implement functions $\texttt{moveRight}(S, r, c)$.

In [17]:
def moveRight(S, r, c):
    "Move the tile left of the blank to the right."
    T = listOfLists(S)
    T[r][c], T[r][c-1] = (T[r][c-1], 0)
    return tupleOfTuples(T)

In [18]:
def moveLeft(S, r, c):
    " Move the tile right of the blank to the left."
    T = listOfLists(S)
    T[r][c], T[r][c+1] = (T[r][c+1], 0)
    return tupleOfTuples(T)

**Exercise 4:**. Implement a function $\texttt{nextStates}(S)$ that takes a state $S$ representet as a tuple of tuple and that computes the set of states that are reachable from $S$ in one step.  Remember that the function $\texttt{findZero}(S)$ has already been defined.

In [19]:
def nextStates(S):
    row, col = findZero(S)
    result = set()
    if col > 0:
        result.add(moveRight(S, row, col))
    if col < 2:
        result.add(moveLeft(S, row, col))
    if row > 0:
        result.add(moveDown(S, row, col))
    if row < 2:
        result.add(moveUp(S, row, col))
    return result

In [20]:
import time

The following computation takes about 10 seconds on my desktop computer, which has an 3,4 GHz Quad-Core Intel Core i5 (7500) Prozessor.  The final number of states touched is $181\,440$.  

In [21]:
timeStart  = time.time()
Path       = findPath(start, goal, nextStates)
timeFinish = time.time()
print(f'The computation took {round((timeFinish-timeStart)*10)/10} seconds.')

  1: Number of states =      4
  2: Number of states =      9
  3: Number of states =     19
  4: Number of states =     33
  5: Number of states =     61
  6: Number of states =    103
  7: Number of states =    183
  8: Number of states =    291
  9: Number of states =    493
 10: Number of states =    771
 11: Number of states =   1295
 12: Number of states =   2021
 13: Number of states =   3369
 14: Number of states =   5173
 15: Number of states =   8456
 16: Number of states =  12649
 17: Number of states =  19971
 18: Number of states =  28567
 19: Number of states =  42497
 20: Number of states =  57210
 21: Number of states =  78931
 22: Number of states =  98758
 23: Number of states = 123890
 24: Number of states = 142087
 25: Number of states = 161065
 26: Number of states = 170994
 27: Number of states = 178353
 28: Number of states = 180434
 29: Number of states = 181312
 30: Number of states = 181438
 31: Number of states = 181440
The computation took 9.4 seconds.


The tuple Path that is a solution to the sliding problem has a length of **32**.  If your path is shorter, then you have to inspect it carefully to identify the problem.

In [22]:
len(Path)

32

## Code for Pretty Printing

In [23]:
def stateToStringList(S):
    "Take a state and transform it into a list of strings."
    result = []
    indent = " " * 4;
    result = [ indent + "+*+*+*+" ]
    for row in (0, 1, 2):
        line = indent + "|"
        for col in (0, 1, 2):
            cell = S[row][col]
            if cell > 0:
                line += str(cell)
            else:
                line += " "
            line += "|"
        result += [ line, indent + "+*+*+*+" ]
    result += [ "" ]
    return result;

In [24]:
def printPath(Path):
    "Take a list of states and print it in a readable fashion."
    for S in Path:
        for line in stateToStringList(S):
            print(line)

In [25]:
printPath(Path)

    +*+*+*+
    |8| |6|
    +*+*+*+
    |5|4|7|
    +*+*+*+
    |2|3|1|
    +*+*+*+

    +*+*+*+
    | |8|6|
    +*+*+*+
    |5|4|7|
    +*+*+*+
    |2|3|1|
    +*+*+*+

    +*+*+*+
    |5|8|6|
    +*+*+*+
    | |4|7|
    +*+*+*+
    |2|3|1|
    +*+*+*+

    +*+*+*+
    |5|8|6|
    +*+*+*+
    |4| |7|
    +*+*+*+
    |2|3|1|
    +*+*+*+

    +*+*+*+
    |5|8|6|
    +*+*+*+
    |4|3|7|
    +*+*+*+
    |2| |1|
    +*+*+*+

    +*+*+*+
    |5|8|6|
    +*+*+*+
    |4|3|7|
    +*+*+*+
    | |2|1|
    +*+*+*+

    +*+*+*+
    |5|8|6|
    +*+*+*+
    | |3|7|
    +*+*+*+
    |4|2|1|
    +*+*+*+

    +*+*+*+
    |5|8|6|
    +*+*+*+
    |3| |7|
    +*+*+*+
    |4|2|1|
    +*+*+*+

    +*+*+*+
    |5| |6|
    +*+*+*+
    |3|8|7|
    +*+*+*+
    |4|2|1|
    +*+*+*+

    +*+*+*+
    |5|6| |
    +*+*+*+
    |3|8|7|
    +*+*+*+
    |4|2|1|
    +*+*+*+

    +*+*+*+
    |5|6|7|
    +*+*+*+
    |3|8| |
    +*+*+*+
    |4|2|1|
    +*+*+*+

    +*+*+*+
    |5|6|7|
    +*+*+*+
    |3|8|1|
    +*+*+*+
    |